In [35]:
!playwright install chromium

In [36]:
import os
import json
import asyncio
import nest_asyncio
import numpy as np
import pandas as pd
import time, random
from tqdm import tqdm
from lxml import html
from playwright.async_api import async_playwright
from concurrent.futures import ThreadPoolExecutor, as_completed

In [37]:
with open('site_urls.json', 'r') as f:
    site_urls = json.load(f)

with open('property_urls.json', 'r') as f:
    property_urls = json.load(f)

In [38]:
site_urls['urls']

['https://www.leopalace21.com',
 'https://www.leopalace21.com/en',
 'https://www.leopalace21.com/en/search/chintai/area',
 'https://www.leopalace21.com/en/properties/chintai/area/PREFECTURE?page=']

In [39]:
property_urls['urls'][:5]

['https://www.leopalace21.com/en/properties/common/hokkaido/ebetsu-shi-01217/light-garden-37080',
 'https://www.leopalace21.com/en/properties/common/hokkaido/ebetsu-shi-01217/montmartre-no-oka-38198',
 'https://www.leopalace21.com/en/properties/common/hokkaido/hakodate-shi-01202/hakodate-22098',
 'https://www.leopalace21.com/en/properties/common/hokkaido/hakodate-shi-01202/himawari-2-23510',
 'https://www.leopalace21.com/en/properties/common/hokkaido/hokuto-shi-01236/dolce-k-y-20774']

NORMAL FUNCTION

In [40]:
def query_details(tree, xpath):
    return

ASYNC FUNCTIONS

In [66]:
async def scrape_data(browser, url):
    page = await browser.new_page()
    url = site_urls['urls'][1] + url
    await page.goto(url, wait_until='domcontentloaded', timeout=60000)
    content = await page.content()
    tree = html.fromstring(content)
    
    details = tree.xpath('//div[@class="page_head-bar__lmpS8"]//text()')
    return details

async def fetch_property(client, url):
    browser = await client.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto(url)
    content = await page.content()
    tree = html.fromstring(content)
    
    tasks = []
    links = tree.xpath('//a[@class="RoomItem_link__RoXCn"]/@href')
    for url in links:
        tasks.append(scrape_data(browser, url))
        
    result = await asyncio.gather(*tasks)
    return result
    
async def main():
    async with async_playwright() as client:
        tasks = []
        for url in property_urls['urls'][:3]:
            tasks.append(fetch_property(client, url))
        
        results = await asyncio.gather(*tasks)
        # flaten_results = [item for sublist in results for item in sublist]
        for i in results:
            print(i)
            

In [ ]:
nest_asyncio.apply()
asyncio.run(main())

['/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/101', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/103', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/104', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/106', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/107', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/201', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/202', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/203', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/204', '/properties/chintai/hokkaido/ebetsu-shi-01217/light-garden-37080/206', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/101', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/103', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/104', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/106', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/107', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/201', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/202', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/203', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/204', '/properties/monthly/hokkaido/ebetsu-shi-01217/light-garden-37080/206']